<a href="https://colab.research.google.com/github/ABHINAV-555/ABHINAV-555/blob/main/spss_final_done_ok_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd

# Step 1: Upload Excel file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Step 2: Read the first row (questions)
question_texts = pd.read_excel(file_name, header=None, nrows=1).iloc[0].tolist()

# Step 3: Read the actual data
df = pd.read_excel(file_name, skiprows=1)

# Step 4: Show column info
print("\n✅ File loaded!")
print("\nAvailable columns:")
for idx, col in enumerate(df.columns):
    print(f"{idx}: {col} — {question_texts[idx]}")

# Step 5: Ask for starting column
start_index = int(input("\n👉 Enter the column index number from which to start frequency analysis: "))
if start_index < 0 or start_index >= len(df.columns):
    raise ValueError("❌ Invalid column index!")

df_selected = df.iloc[:, start_index:]
question_texts_selected = question_texts[start_index:]

# Step 6: Build output
output_rows = []
output_rows.append(["STATEMENT (N = {})".format(len(df)), "FREQUENCY", "PERCENTAGE", "FREQUENCY", "PERCENTAGE"])

for idx, col in enumerate(df_selected.columns):
    try:
        col_data = df[col].dropna()
        question_label = str(question_texts_selected[idx]).replace('\n', ' ').replace('\r', ' ').replace('\xa0', ' ').strip()

        # ⏳ Skip datetime columns
        if pd.api.types.is_datetime64_any_dtype(col_data) or col_data.astype(str).str.match(r"\d{4}-\d{2}-\d{2}|\d{2}/\d{2}/\d{4}").any():
            print(f"⏭️ Skipping column '{col}' — detected as date/time.")
            continue

        output_rows.append([f"Question: {question_label}", "", "", "", ""])

        if pd.api.types.is_numeric_dtype(col_data):
            # 🟦 Numeric summary
            mean = round(col_data.mean(), 2)
            median = round(col_data.median(), 2)
            mode = col_data.mode()
            mode_val = mode.iloc[0] if not mode.empty else "N/A"

            output_rows.append(["Mean", mean, "", "", ""])
            output_rows.append(["Median", median, "", "", ""])
            output_rows.append(["Mode", mode_val, "", "", ""])
            output_rows.append(["Total (Valid)", len(col_data), "", "", ""])  # Total valid count
            output_rows.append(["", "", "", "", ""])

        else:
            # 🟨 Categorical frequency
            freq_table = col_data.value_counts(dropna=False).reset_index()
            freq_table.columns = ['Category', 'Frequency']
            freq_table['Percentage'] = round((freq_table['Frequency'] / len(df)) * 100, 2)

            for _, row in freq_table.iterrows():
                label = str(row['Category']) if pd.notna(row['Category']) else "Missing"
                output_rows.append([label, row['Frequency'], row['Percentage'], "", ""])

            # ➕ Add Total row
            total_freq = freq_table['Frequency'].sum()
            total_pct = freq_table['Percentage'].sum()
            output_rows.append(["Total", total_freq, total_pct, "", ""])
            output_rows.append(["", "", "", "", ""])

    except Exception as e:
        print(f"⚠️ Skipping column '{col}' due to error: {e}")

# Step 7: Export to Excel
output_df = pd.DataFrame(output_rows)
output_file = "final_frequency_and_summary_with_totals.xlsx"
output_df.to_excel(output_file, index=False, header=False)

print(f"\n✅ Report with totals generated: {output_file}")
files.download(output_file)
